In [33]:
import os
from astropy.io import fits
from astropy.table import Table
import lsdb
from pathlib import Path
from upath import UPath
from astropy.wcs import WCS
import numpy as np
import pandas as pd
print('Imported libaries.')

Imported libaries.


In [4]:
gaia_pm_greater_1000 = lsdb.read_hats('../../../catalogs/gaia_dr3_pm_greater_1000').query('phot_g_mean_mag > 19 or phot_g_mean_mag.isna()').compute()
gaia_pm_greater_1000

,source_id,ra,dec,pmra,pmdec,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag
_healpix_29,,,,,,,,
72355622838789655,144711230753602048,68.900732,21.250996,860.716142,-959.20791,19.120424,21.595911,17.604784
279061151160369797,558122277038055808,83.298901,82.771826,2038.794128,-1662.957624,<NA>,<NA>,18.49547
465411006967832544,930822015518055808,127.527141,48.476428,-1016.564527,-774.971624,20.710526,21.585167,19.04894
794625047801153335,1589250088362827776,228.747226,48.801312,-938.043889,1464.629578,19.241777,21.351543,17.604332
847595661249829565,1695191332232990720,226.702636,70.461619,-1194.179224,1042.194214,20.199528,21.771992,18.33195
1141806263107329672,2283612503305198208,321.76063,76.299039,756.414332,822.46256,19.926235,21.45336,18.204111
1286450513388042213,2572901021957789568,24.24125,9.56307,1238.243937,-16.155572,19.461197,21.107285,17.640982
1819280488241997640,3638560954457699200,202.951993,-1.285494,-414.209178,-1041.278255,20.534954,21.39854,18.914696
1849489717977963240,3698979462002285824,180.986807,0.262663,-1219.590296,-282.117684,19.133556,20.761082,17.564669


In [14]:
def getimages(ra,dec,filters="grizy"):
    
    """Query ps1filenames.py service to get a list of images
    
    ra, dec = position in degrees
    size = image size in pixels (0.25 arcsec/pixel)
    filters = string with filters to include
    Returns a table with the results
    """
    
    service = "https://ps1images.stsci.edu/cgi-bin/ps1filenames.py"
    url = f"{service}?ra={ra}&dec={dec}&filters={filters}&type=warp,warp.wt,warp.mask&format=fits&size=40"
    print(url)
    table = Table.read(url, format='ascii')
    return table

In [37]:
def combine_fits_files(source_id_gaia, 
                       ra_gaia, dec_gaia, pm_gaia, 
                       gmag_gaia, bpmag_gaia, rpmag_gaia, row):
    
    # We skip variance and masks to not double or triple count the same image
    if (('wt' == row['filename'].split('.')[10]) or
        ('mask' == row['filename'].split('.')[10])):
        return

    sci_filename = row['filename']
    mask_filename = '.'.join(sci_filename.split('.')[:10]) + '.mask.fits'
    var_filename = '.'.join(sci_filename.split('.')[:10]) + '.wt.fits'
    
    sci_file_path = UPath('./ps1_images_combined_v2' + sci_filename)
    mask_file_path = UPath('./ps1_images_combined_v2' + mask_filename)
    var_file_path = UPath('./ps1_images_combined_v2' + var_filename)

    sci_url = UPath('https://ps1images.stsci.edu' + sci_filename)
    mask_url = UPath('https://ps1images.stsci.edu' + mask_filename)
    var_url = UPath('https://ps1images.stsci.edu' + var_filename)
    
    sci_file_path.parent.mkdir(exist_ok=True, parents=True)
    mask_file_path.parent.mkdir(exist_ok=True, parents=True)
    var_file_path.parent.mkdir(exist_ok=True, parents=True)

    # If not already downloaded, download image files from cutout service.
    if not sci_file_path.exists():
        with sci_url.open('rb') as src, sci_file_path.open('wb') as dest:
            dest.write(src.read())
    if not mask_file_path.exists():
        with mask_url.open('rb') as src, mask_file_path.open('wb') as dest:
            dest.write(src.read())
    if not var_file_path.exists():
        with var_url.open('rb') as src, var_file_path.open('wb') as dest:
            dest.write(src.read())

    output_filename = '.'.join(sci_filename.split('.')[:10]) + '.combined.fits'
    output_file_path = UPath('./ps1_images_combined_v2' + output_filename)
    
    # Load data
    science_data = fits.getdata(sci_file_path, 1)
    hdr = fits.getheader(sci_file_path, 1)
    wcs = WCS(hdr)
    mask_data = fits.getdata(mask_file_path, 1)
    # Uninvert variance
    var_data = (1 / fits.getdata(var_file_path, 1))

    # Wipe most data from header, and add gaia stats + link
    hdr = wcs.to_header()
    hdr['id_gaia'] = source_id_gaia
    hdr['ra_gaia'] = ra_gaia
    hdr['dec_gaia'] = dec_gaia
    hdr['pm_gaia'] = pm_gaia
    hdr['gmag'] = gmag_gaia if not pd.isna(gmag_gaia) else 'nan'
    hdr['bpmag'] = bpmag_gaia if not pd.isna(bpmag_gaia) else 'nan'
    hdr['rpmag'] = rpmag_gaia if not pd.isna(rpmag_gaia) else 'nan'
    hdr['download_link'] = str(sci_url)

    # Create HDUs in order: SCIENCE, MASK, VARIANCE
    primary_hdu = fits.PrimaryHDU(header=hdr)
    science_hdu = fits.ImageHDU(data=science_data, header=hdr, name='SCIENCE')
    mask_hdu = fits.ImageHDU(data=mask_data, name='MASK')
    var_hdu = fits.ImageHDU(data=var_data, name='VARIANCE')

    # Combine and write
    hdul = fits.HDUList([primary_hdu, science_hdu, mask_hdu, var_hdu])
    hdul.writeto(output_file_path, overwrite=True)

    # Delete original files
    os.remove(sci_file_path)
    os.remove(mask_file_path)
    os.remove(var_file_path)


In [38]:
for index, row_gaia in gaia_pm_greater_1000.iterrows():
    # 0.5 arcsecs per pixel, so 10 arcsec images
    i_images = getimages(row_gaia['ra'], row_gaia['dec'], filters='i')
    pm_gaia = (row['pmra']**2 + row['pmdec']**2)**0.5
    for idx, row_table in enumerate(i_images):
        combine_fits_files(row_gaia['source_id'], 
                       row_gaia['ra'], row_gaia['dec'], pm_gaia, 
                       row_gaia['phot_g_mean_mag'], row_gaia['phot_bp_mean_mag'], 
                       row_gaia['phot_rp_mean_mag'], row_table)

https://ps1images.stsci.edu/cgi-bin/ps1filenames.py?ra=68.90073185475295&dec=21.250996459036724&filters=i&type=warp,warp.wt,warp.mask&format=fits&size=40


this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]


https://ps1images.stsci.edu/cgi-bin/ps1filenames.py?ra=83.29890115430591&dec=82.77182612147968&filters=i&type=warp,warp.wt,warp.mask&format=fits&size=40


https://ps1images.stsci.edu/cgi-bin/ps1filenames.py?ra=127.52714090400372&dec=48.47642773218372&filters=i&type=warp,warp.wt,warp.mask&format=fits&size=40


https://ps1images.stsci.edu/cgi-bin/ps1filenames.py?ra=228.7472256453729&dec=48.8013123316322&filters=i&type=warp,warp.wt,warp.mask&format=fits&size=40


...

https://ps1images.stsci.edu/cgi-bin/ps1filenames.py?ra=180.98680696477484&dec=0.2626631473947131&filters=i&type=warp,warp.wt,warp.mask&format=fits&size=40


https://ps1images.stsci.edu/cgi-bin/ps1filenames.py?ra=179.58893132321765&dec=4.57989695872048&filters=i&type=warp,warp.wt,warp.mask&format=fits&size=40


https://ps1images.stsci.edu/cgi-bin/ps1filenames.py?ra=57.79123387834184&dec=-56.456614645499855&filters=i&type=warp,warp.wt,warp.mask&format=fits&size=40
https://ps1images.stsci.edu/cgi-bin/ps1filenames.py?ra=30.50496782970952&dec=-31.614258423805172&filters=i&type=warp,warp.wt,warp.mask&format=fits&size=40


https://ps1images.stsci.edu/cgi-bin/ps1filenames.py?ra=162.48921494407233&dec=-74.00780549813459&filters=i&type=warp,warp.wt,warp.mask&format=fits&size=40
https://ps1images.stsci.edu/cgi-bin/ps1filenames.py?ra=124.37351805519964&dec=-61.91613022264878&filters=i&type=warp,warp.wt,warp.mask&format=fits&size=40
https://ps1images.stsci.edu/cgi-bin/ps1filenames.py?ra=171.65484488705087&dec=-50.06323211342603&filters=i&type=warp,warp.wt,warp.mask&format=fits&size=40
https://ps1images.stsci.edu/cgi-bin/ps1filenames.py?ra=125.58069426513862&dec=-12.823456020671188&filters=i&type=warp,warp.wt,warp.mask&format=fits&size=40


https://ps1images.stsci.edu/cgi-bin/ps1filenames.py?ra=194.1812246704677&dec=-62.03559791438614&filters=i&type=warp,warp.wt,warp.mask&format=fits&size=40
https://ps1images.stsci.edu/cgi-bin/ps1filenames.py?ra=194.1707401155348&dec=-62.03441920233982&filters=i&type=warp,warp.wt,warp.mask&format=fits&size=40


In [13]:
print('Executing fits combine')
combine_fits_files("ps1_images_combined")
print('Done')

Executing fits combine
ps1_images_combined []
ps1_images_combined/rings.v3.skycell []
ps1_images_combined/rings.v3.skycell/1456 []
ps1_images_combined/rings.v3.skycell/1456/011 ['rings.v3.skycell.1456.011.wrp.i.57028_60833.mask.fits', 'rings.v3.skycell.1456.011.wrp.i.57001_60133.wt.fits', 'rings.v3.skycell.1456.011.wrp.i.56794_37596.mask.fits', 'rings.v3.skycell.1456.011.wrp.g.55630_53887.mask.fits', 'rings.v3.skycell.1456.011.wrp.i.57001_58944.wt.fits', 'rings.v3.skycell.1456.011.wrp.i.56708_44160.wt.fits', 'rings.v3.skycell.1456.011.wrp.i.57053_55946.wt.fits', 'rings.v3.skycell.1456.011.wrp.i.57053_53593.wt.fits', 'rings.v3.skycell.1456.011.wrp.i.55971_60466.mask.fits', 'rings.v3.skycell.1456.011.wrp.i.56698_50058.fits', 'rings.v3.skycell.1456.011.wrp.i.56708_45375.fits', 'rings.v3.skycell.1456.011.wrp.i.56708_47831.mask.fits', 'rings.v3.skycell.1456.011.wrp.i.57053_53593.fits', 'rings.v3.skycell.1456.011.wrp.i.57001_60133.mask.fits', 'rings.v3.skycell.1456.011.wrp.i.57001_58944.mask

Combined and deleted originals: ps1_images_combined/rings.v3.skycell/1456/011/rings.v3.skycell.1456.011.wrp.i.56698_50058.combined.fits
Combined and deleted originals: ps1_images_combined/rings.v3.skycell/1456/011/rings.v3.skycell.1456.011.wrp.i.56708_45375.combined.fits
Combined and deleted originals: ps1_images_combined/rings.v3.skycell/1456/011/rings.v3.skycell.1456.011.wrp.i.57053_53593.combined.fits
Combined and deleted originals: ps1_images_combined/rings.v3.skycell/1456/011/rings.v3.skycell.1456.011.wrp.i.57028_62272.combined.fits
Combined and deleted originals: ps1_images_combined/rings.v3.skycell/1456/011/rings.v3.skycell.1456.011.wrp.i.56794_36425.combined.fits
Combined and deleted originals: ps1_images_combined/rings.v3.skycell/1456/011/rings.v3.skycell.1456.011.wrp.i.56668_54869.combined.fits
Combined and deleted originals: ps1_images_combined/rings.v3.skycell/1456/011/rings.v3.skycell.1456.011.wrp.i.56400_29932.combined.fits
Combined and deleted originals: ps1_images_combi